# Using Gemini in AutoGen with Other LLMs

You don't need to handle OpenAI or Google's GenAI packages. AutoGen already handled all of these for you.

You can just create different agents with different backend LLM with assistant agent, and all models/agents are at your fingertip.


## Main Distinctions
- Gemini does not have the "system_message" field (correct me if I am wrong). So, it's instruction following skills are not as strong as GPTs.


Sample OAI_CONFIG_LIST 

```python
[
    {
        "model": "gpt-35-turbo",
        "api_key": "your OpenAI Key goes here",
        "base_url": "https://tnrllmproxy.azurewebsites.net/v1",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "gpt-4-vision-preview",
        "api_key": "your OpenAI Key goes here",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "dalle",
        "api_key": "your OpenAI Key goes here",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "gemini-pro",
        "api_key": "your Google's GenAI Key goes here",
        "api_type": "google"
    },
    {
        "model": "gemini-pro-vision",
        "api_key": "your Google's GenAI Key goes here",
        "api_type": "google"
    }
]
```

### Before everything starts, install AutoGen with the `gemini` option
```bash
pip install "pyautogen[gemini]~=0.2.0b4"
```

In [1]:
import requests
import json
import pdb
import os
import re

from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent

from autogen.agentchat.contrib.img_utils import get_image_data, _to_pil
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

from termcolor import colored
import random

from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang


In [2]:
import os
import PIL
import matplotlib.pyplot as plt

In [3]:

config_list_4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview"],
    },
)

config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)

config_list_gemini_lmm = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro-vision"],
    },
)

## Gemini Assitant


In [4]:
assistant = AssistantAgent("assistant", 
                           llm_config={"config_list": config_list_gemini, "seed": 42}, 
                           max_consecutive_auto_reply=3)
# print(assistant.system_message)

user_proxy = UserProxyAgent("user_proxy", 
                            code_execution_config={"work_dir": "coding", "use_docker": False}, 
                            human_input_mode="NEVER", 
                           is_termination_msg = lambda x: content_str(x.get("content")).find("TERMINATE") >= 0)

user_proxy.initiate_chat(assistant, message="Sort the array with Bubble Sort: [4, 1, 3, 2]")

user_proxy (to assistant):

Sort the array with Bubble Sort: [4, 1, 3, 2]

--------------------------------------------------------------------------------
assistant (to user_proxy):

```python
# filename: bubble_sort.py
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

if __name__ == "__main__":
    arr = [4, 1, 3, 2]
    print("Unsorted array: ", arr)
    sorted_arr = bubble_sort(arr)
    print("Sorted array: ", sorted_arr)
```

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Unsorted array:  [4, 1, 3, 2]
Sorted array:  [1, 2, 3, 4]


--------------------------------------------------------------------------------
assistant (to user_proxy):

TERMINATE

--------------

## Agent Collaboration and Interactions



In [5]:
gpt = AssistantAgent("GPT-4", 
                     system_message="""You should ask weird, tricky, and concise questions. 
Ask the next question based on (by evolving) the previous one.""",
                       llm_config={"config_list": config_list_gpt4, "seed": 42}, 
                       max_consecutive_auto_reply=3)

gemini = AssistantAgent("Gemini-Pro", 
                     system_message="""Always answer questions within one sentence. """,
#                      system_message="answer:",
                       llm_config={"config_list": config_list_gemini, "seed": 42}, 
                       max_consecutive_auto_reply=4)



gpt.initiate_chat(gemini, message="Do Transformers buy auto insurance or health insurance?")

GPT-4 (to Gemini-Pro):

Do Transformers buy auto insurance or health insurance?

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

Transformers, being fictional robots, do not require auto or health insurance.

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

If Transformers were real and required insurance, would their coverage be a specialized form of property insurance instead of typical auto or health insurance?

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

Yes, if Transformers were real and required insurance, their coverage would likely be a specialized form of property insurance designed specifically for sentient robotic beings.

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

Considering their dual nature as vehicles and sentient beings, could they poten

In [6]:
gpt.initiate_chat(gemini, message="Should Spider-Man invest in 401K?")


GPT-4 (to Gemini-Pro):

Should Spider-Man invest in 401K?

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

Yes, Spider-Man should invest in a 401K to secure his financial future.

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

Given Spider-Man's age and potential lifespan, what would be an appropriate asset allocation for his 401K?

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

An appropriate asset allocation for Spider-Man's 401K, given his age and potential lifespan, would be 80% stocks and 20% bonds.

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

Considering Spider-Man's high-risk occupation, should he adjust this allocation to accommodate for potential periods of unemployment or disability?

--------------------------------------------------------------

Let's switch position. Now, Gemini is the question raiser. 

This time, Gemini could not follow the system instruction well or evolve questions, because the Gemini does not handle system messages similar to GPTs.

In [7]:
gpt = AssistantAgent("GPT-4", 
                     system_message="""Always answer questions within one sentence. """,
                       llm_config={"config_list": config_list_gpt4, "seed": 42}, 
                       max_consecutive_auto_reply=3)

gemini = AssistantAgent("Gemini-Pro", 
                     system_message="""You should ask weird, tricky, and concise questions. 
Ask the next question based on (by evolving) the previous one.""",
                       llm_config={"config_list": config_list_gemini, "seed": 42}, 
                       max_consecutive_auto_reply=4)

gemini.initiate_chat(gpt, message="Do Transformers buy auto insurance or health insurance?")

Gemini-Pro (to GPT-4):

Do Transformers buy auto insurance or health insurance?

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

Transformers are fictional robots, so they don't buy insurance, but if they did, they might need a special kind of insurance that combines aspects of both auto and health insurance.

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

If the sun was a giant ball of cotton candy, would it be possible to eat it or would it melt too quickly?

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

If the sun were a giant ball of cotton candy, it would not be possible to eat it because it would either disintegrate due to the vacuum of space or melt instantly from the residual heat.

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

If cats dreamt of bei

In [8]:
gemini.initiate_chat(gpt, message="Should Spider-Man invest in 401K?")


Gemini-Pro (to GPT-4):

Should Spider-Man invest in 401K?

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

As a fictional character, Spider-Man does not require a 401(k), but his alter ego, if employed with such benefits, might consider it for retirement savings.

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

If the sun sneezes, does it become stars?

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

No, a sneeze is a biological function that the sun, being a star, cannot perform.

--------------------------------------------------------------------------------
Gemini-Pro (to GPT-4):

If you could have any animal as your neighbor, who would it be and why a giraffe?

--------------------------------------------------------------------------------
GPT-4 (to Gemini-Pro):

A giraffe could be an interesting neighbor due to

## Gemini RAG

## Gemini Multimodal

You can create multimodal agent for Gemini the same way as the GPT-4V and LLaVA

## GPT  v.s. Gemini Arena

Do you remember AutoGen can ask LLMs to play chess. Here, we create an arena to allow GPT and Gemini fight together.